# CROSS-REPRESENTATION TRANSFERABILITY OF ADVERSARIAL PERTURBATIONS: FROM SPECTROGRAMS TO AUDIO WAVEFORMS


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os, sys
import soundfile as sf
import IPython.display

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from keras import regularizers

import os, sys
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten
from keras.callbacks import TensorBoard
from keras.utils import np_utils, to_categorical
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
import keras.initializers as init

os.environ["CUDA_VISIBLE_DEVICES"]="3"
import tensorflow as tf
config = tf.ConfigProto( )
config.gpu_options.allow_growth = True
sess   = tf.Session(config=config)
import keras.backend.tensorflow_backend as tf_bkend
tf_bkend.set_session(sess)

Using TensorFlow backend.


In [2]:
# Controling Hyperparameters
nb_classes = 10
frame_size = 110250
sr = 22050

In [3]:
#Indicate folds
train_fold = [2, 3]
test_fold = 1

str_train_fold = "fold"+str(train_fold[0])+"-"+str(train_fold[1])
str_test_fold  = "fold"+str(test_fold)

In [4]:
#Load original wave paths
original_wave_list  = []
base_path = "gtzan_3f_Win_110250_75/"

path = base_path+"fold"+str(test_fold)

j = 0
for path, subdirs, files in os.walk(path):
    for name in files:
        original_wave_list.append(os.path.join(path, name))
        j += 1
print("Original waves processed: "+"fold "+str(test_fold)+" - "+str(j) )

Original waves processed: fold 1 - 7138


In [5]:
def model_generator_mf():
    
    from keras.layers      import Input, Dense, Conv2D, AveragePooling1D, LeakyReLU, MaxPool2D, Flatten
    from keras.layers.core import Dropout
    from keras.models      import Model
    from keras             import initializers, optimizers, regularizers
    from keras.callbacks   import ModelCheckpoint
    from keras.utils       import multi_gpu_model
    
    from keras.layers.normalization import BatchNormalization
        
    import keras.initializers as init
    
    from kapre.utils          import Normalization2D
    from kapre.augmentation   import AdditiveNoise    
    
    sr = 22050
    
    inp   = Input(shape = (f, g, 1)) 
    #----------------------
    conv1  = Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu')(inp)
    norm1  = BatchNormalization()(conv1)
    #----------------------
    conv2  = Conv2D(filters = 32, kernel_size = (3, 3) )(norm1)
    act2   = LeakyReLU(alpha = 0.2)(conv2)
    pool2  = MaxPool2D(pool_size = 2, strides = 2)(act2)
    drop2  = Dropout(0.05)(pool2)
    #----------------------
    conv3  = Conv2D(filters = 64, kernel_size = (3, 3) )(drop2)
    act3   = LeakyReLU(alpha = 0.2)(conv3)
    #----------------------
    conv4  = Conv2D(filters = 64, kernel_size = (3, 3) )(act3)
    act4   = LeakyReLU(alpha = 0.2)(conv4)
    pool4  = MaxPool2D(pool_size = 4, strides = 2)(act4)
    #----------------------
    flat   = Flatten()(pool4) 
    #----------------------    
    dense3 = Dense(1024, activation='relu', kernel_initializer = initializers.glorot_uniform(seed = 0))(flat)
    drop3  = Dropout(0.80)(dense3)    
    #----------------------
    dense4 = Dense(nb_classes, activation='softmax')(drop3)
    #----------------------

    model  = Model(inp, dense4)

    model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizers.Adadelta(lr = 1.0, rho = 0.95, epsilon = 1e-08, decay = 0.0),
                  metrics = ['accuracy'] )
    
    model.summary()
    
    return model

In [6]:
X_mf = np.load( "folds_mf/2_GTzan_Xs_test_"+str_test_fold+"_110250_75_frozen.npy" )

f = X_mf.shape[1]
g = X_mf.shape[2]

# Adapt 1D data to 2D CNN
X_mf = np.squeeze( X_mf )
X_mf = np.expand_dims( X_mf, axis = 3 )

print(X_mf.shape)

(7138, 64, 431, 1)


In [7]:
# Re-generating the model 
model_mf = model_generator_mf()
path = str( "weights/weights_3_GTzan_3f_"+str_train_fold+"_20p_110250_75_frozen.best.hdf5" )
model_mf.load_weights(path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 431, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 429, 32)       320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 62, 429, 32)       128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 427, 32)       9248      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 60, 427, 32)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 213, 32)       0         
___________________________________________

In [8]:
def model_generator_stft():
    
    from keras.layers      import Input, Dense, Conv2D, AveragePooling1D, LeakyReLU, MaxPool2D, Flatten
    from keras.layers.core import Dropout
    from keras.models      import Model
    from keras             import initializers, optimizers, regularizers
    from keras.callbacks   import ModelCheckpoint
    from keras.utils       import multi_gpu_model
    
    from keras.layers.normalization import BatchNormalization
        
    import keras.initializers as init

    from kapre.utils          import Normalization2D
    from kapre.augmentation   import AdditiveNoise    
    
    sr = 22050
    
    inp   = Input(shape = (f, g, 1)) 
    #----------------------
    conv1  = Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu')(inp)
    norm1  = BatchNormalization()(conv1)
    #----------------------
    conv2  = Conv2D(filters = 32, kernel_size = (3, 3) )(norm1)
    act2   = LeakyReLU(alpha = 0.2)(conv2)
    pool2  = MaxPool2D(pool_size = 2, strides = 2)(act2)
    drop2  = Dropout(0.05)(pool2)
    #----------------------
    conv3  = Conv2D(filters = 64, kernel_size = (3, 3) )(drop2)
    act3   = LeakyReLU(alpha = 0.2)(conv3)
    #----------------------
    conv4  = Conv2D(filters = 64, kernel_size = (3, 3) )(act3)
    act4   = LeakyReLU(alpha = 0.2)(conv4)
    pool4  = MaxPool2D(pool_size = 4, strides = 2)(act4)
    #----------------------
    flat   = Flatten()(pool4)  
    #----------------------    
    dense3 = Dense(512, activation='relu', kernel_initializer = initializers.glorot_uniform(seed = 0))(flat)
    drop3  = Dropout(0.80)(dense3)    
    #----------------------
    dense4 = Dense(nb_classes, activation='softmax')(drop3)
    #----------------------

    model  = Model(inp, dense4)

    model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizers.Adadelta(lr = 1.0, rho = 0.95, epsilon = 1e-08, decay = 0.0),
                  metrics = ['accuracy'] )
    
    model.summary()
    
    return model

In [9]:
X_stft      = np.load( "folds_stft/2_GTzan_Xs_test_"+str_test_fold+"_110250_75_frozen_stft.npy" )
X_stft_BIM  = np.load( "folds_stft/2_GTzan_Xs_BIM_"+str_test_fold+"_110250_75_frozen_stft.npy" )
X_stft_FGSM = np.load( "folds_stft/2_GTzan_Xs_FGSM_"+str_test_fold+"_110250_75_frozen_stft.npy" )

f = X_stft.shape[1]
g = X_stft.shape[2]

In [10]:
# Adapt 1D data to 2D CNN
X_stft = np.squeeze( X_stft )
X_stft = np.expand_dims( X_stft, axis = 3 )

X_stft_BIM = np.squeeze( X_stft_BIM )
X_stft_BIM = np.expand_dims( X_stft_BIM, axis = 3 )

X_stft_FGSM = np.squeeze( X_stft_FGSM )
X_stft_FGSM = np.expand_dims( X_stft_FGSM, axis = 3 )

print(X_stft.shape)

(7138, 257, 431, 1)


In [11]:
# Re-generating the model 
model_stft = model_generator_stft()
path = str( "weights/weights_3_GTzan_3f_"+str_train_fold+"_20p_110250_75_frozen_stft.best.hdf5" )
model_stft.load_weights(path)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 257, 431, 1)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 255, 429, 32)      320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 255, 429, 32)      128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 253, 427, 32)      9248      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 253, 427, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 126, 213, 32)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 126, 213, 32)      0   

In [12]:
choices = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5,
           'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

In [16]:
def guess_genre(y_proba):
    
    j = 0
    for i in y_proba[0]:
        if i == np.max(y_proba):
            break;
        else: j+=1;       
    for genre, key in choices.items():
        if key == j: return (genre)
    return

In [25]:
for j in range (0, 7130):
    
    print("\n" + "j: " + str(j) + " " + str(original_wave_list[j]))

    #y_proba_mf   = model_mf.predict(X_mf[j:j+1])
    y_proba_stft = model_stft.predict(X_stft[j:j+1])
    y_proba_stft_BIM  = model_stft.predict(X_stft_BIM[j:j+1])
    y_proba_stft_FGSM = model_stft.predict(X_stft_FGSM[j:j+1])

    print("NONE - Predicted genre: " + str(guess_genre(y_proba_stft)))
    print("BIM - Predicted genre: " + str(guess_genre(y_proba_stft_BIM)))
    print("FGSM - Predicted genre: " + str(guess_genre(y_proba_stft_FGSM)))


j: 0 gtzan_3f_Win_110250_75/fold1/metal/689_20.wav
NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 1 gtzan_3f_Win_110250_75/fold1/metal/677_6.wav
NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 2 gtzan_3f_Win_110250_75/fold1/metal/678_13.wav
NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 3 gtzan_3f_Win_110250_75/fold1/metal/692_5.wav
NONE - Predicted genre: disco
BIM - Predicted genre: metal
FGSM - Predicted genre: hiphop

j: 4 gtzan_3f_Win_110250_75/fold1/metal/688_7.wav
NONE - Predicted genre: metal
BIM - Predicted genre: pop
FGSM - Predicted genre: hiphop

j: 5 gtzan_3f_Win_110250_75/fold1/metal/686_13.wav
NONE - Predicted genre: metal
BIM - Predicted genre: hiphop
FGSM - Predicted genre: hiphop

j: 6 gtzan_3f_Win_110250_75/fold1/metal/683_5.wav
NONE - Predicted genre: classical
BIM - Predicted genre: pop
FGSM - Predicted genre: hiphop

j: 7 gtzan_3f_Win

NONE - Predicted genre: metal
BIM - Predicted genre: pop
FGSM - Predicted genre: pop

j: 59 gtzan_3f_Win_110250_75/fold1/metal/673_10.wav
NONE - Predicted genre: metal
BIM - Predicted genre: pop
FGSM - Predicted genre: pop

j: 60 gtzan_3f_Win_110250_75/fold1/metal/694_21.wav
NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 61 gtzan_3f_Win_110250_75/fold1/metal/676_16.wav
NONE - Predicted genre: rock
BIM - Predicted genre: metal
FGSM - Predicted genre: metal

j: 62 gtzan_3f_Win_110250_75/fold1/metal/671_17.wav
NONE - Predicted genre: metal
BIM - Predicted genre: pop
FGSM - Predicted genre: pop

j: 63 gtzan_3f_Win_110250_75/fold1/metal/678_14.wav
NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 64 gtzan_3f_Win_110250_75/fold1/metal/692_6.wav
NONE - Predicted genre: disco
BIM - Predicted genre: metal
FGSM - Predicted genre: hiphop

j: 65 gtzan_3f_Win_110250_75/fold1/metal/693_13.wav
NONE - Predicted genre:

NONE - Predicted genre: hiphop
BIM - Predicted genre: blues
FGSM - Predicted genre: pop

j: 119 gtzan_3f_Win_110250_75/fold1/metal/666_18.wav
NONE - Predicted genre: metal
BIM - Predicted genre: pop
FGSM - Predicted genre: hiphop

j: 120 gtzan_3f_Win_110250_75/fold1/metal/669_20.wav
NONE - Predicted genre: country
BIM - Predicted genre: rock
FGSM - Predicted genre: pop

j: 121 gtzan_3f_Win_110250_75/fold1/metal/680_7.wav
NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 122 gtzan_3f_Win_110250_75/fold1/metal/693_12.wav
NONE - Predicted genre: metal
BIM - Predicted genre: pop
FGSM - Predicted genre: pop

j: 123 gtzan_3f_Win_110250_75/fold1/metal/665_4.wav
NONE - Predicted genre: metal
BIM - Predicted genre: disco
FGSM - Predicted genre: disco

j: 124 gtzan_3f_Win_110250_75/fold1/metal/681_20.wav
NONE - Predicted genre: rock
BIM - Predicted genre: disco
FGSM - Predicted genre: hiphop

j: 125 gtzan_3f_Win_110250_75/fold1/metal/666_21.wav
NONE - Pre

NONE - Predicted genre: metal
BIM - Predicted genre: reggae
FGSM - Predicted genre: hiphop

j: 179 gtzan_3f_Win_110250_75/fold1/metal/679_17.wav
NONE - Predicted genre: metal
BIM - Predicted genre: pop
FGSM - Predicted genre: pop

j: 180 gtzan_3f_Win_110250_75/fold1/metal/670_18.wav
NONE - Predicted genre: metal
BIM - Predicted genre: reggae
FGSM - Predicted genre: hiphop

j: 181 gtzan_3f_Win_110250_75/fold1/metal/684_15.wav
NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: disco

j: 182 gtzan_3f_Win_110250_75/fold1/metal/667_4.wav
NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: country

j: 183 gtzan_3f_Win_110250_75/fold1/metal/674_3.wav
NONE - Predicted genre: metal
BIM - Predicted genre: disco
FGSM - Predicted genre: disco

j: 184 gtzan_3f_Win_110250_75/fold1/metal/663_4.wav
NONE - Predicted genre: metal
BIM - Predicted genre: hiphop
FGSM - Predicted genre: hiphop

j: 185 gtzan_3f_Win_110250_75/fold1/metal/690_9.wav
N

NONE - Predicted genre: metal
BIM - Predicted genre: pop
FGSM - Predicted genre: pop

j: 239 gtzan_3f_Win_110250_75/fold1/metal/668_3.wav
NONE - Predicted genre: metal
BIM - Predicted genre: country
FGSM - Predicted genre: pop

j: 240 gtzan_3f_Win_110250_75/fold1/metal/689_1.wav
NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 241 gtzan_3f_Win_110250_75/fold1/metal/661_20.wav
NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 242 gtzan_3f_Win_110250_75/fold1/metal/666_6.wav
NONE - Predicted genre: metal
BIM - Predicted genre: pop
FGSM - Predicted genre: hiphop

j: 243 gtzan_3f_Win_110250_75/fold1/metal/662_1.wav
NONE - Predicted genre: disco
BIM - Predicted genre: metal
FGSM - Predicted genre: hiphop

j: 244 gtzan_3f_Win_110250_75/fold1/metal/663_6.wav
NONE - Predicted genre: metal
BIM - Predicted genre: hiphop
FGSM - Predicted genre: hiphop

j: 245 gtzan_3f_Win_110250_75/fold1/metal/693_9.wav
NONE - Pred

...

NONE - Predicted genre: reggae
BIM - Predicted genre: hiphop
FGSM - Predicted genre: hiphop

j: 6869 gtzan_3f_Win_110250_75/fold1/rock/839_21.wav
NONE - Predicted genre: reggae
BIM - Predicted genre: disco
FGSM - Predicted genre: disco

j: 6870 gtzan_3f_Win_110250_75/fold1/rock/859_20.wav
NONE - Predicted genre: country
BIM - Predicted genre: classical
FGSM - Predicted genre: classical

j: 6871 gtzan_3f_Win_110250_75/fold1/rock/832_14.wav
NONE - Predicted genre: rock
BIM - Predicted genre: country
FGSM - Predicted genre: country

j: 6872 gtzan_3f_Win_110250_75/fold1/rock/831_3.wav
NONE - Predicted genre: country
BIM - Predicted genre: jazz
FGSM - Predicted genre: jazz

j: 6873 gtzan_3f_Win_110250_75/fold1/rock/841_20.wav
NONE - Predicted genre: reggae
BIM - Predicted genre: blues
FGSM - Predicted genre: hiphop

j: 6874 gtzan_3f_Win_110250_75/fold1/rock/860_7.wav
NONE - Predicted genre: country
BIM - Predicted genre: jazz
FGSM - Predicted genre: jazz

j: 6875 gtzan_3f_Win_110250_75/fold

NONE - Predicted genre: rock
BIM - Predicted genre: country
FGSM - Predicted genre: country

j: 6927 gtzan_3f_Win_110250_75/fold1/rock/845_3.wav
NONE - Predicted genre: rock
BIM - Predicted genre: country
FGSM - Predicted genre: country

j: 6928 gtzan_3f_Win_110250_75/fold1/rock/841_16.wav
NONE - Predicted genre: reggae
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 6929 gtzan_3f_Win_110250_75/fold1/rock/844_9.wav
NONE - Predicted genre: rock
BIM - Predicted genre: blues
FGSM - Predicted genre: hiphop

j: 6930 gtzan_3f_Win_110250_75/fold1/rock/841_19.wav
NONE - Predicted genre: reggae
BIM - Predicted genre: disco
FGSM - Predicted genre: disco

j: 6931 gtzan_3f_Win_110250_75/fold1/rock/846_8.wav
NONE - Predicted genre: country
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 6932 gtzan_3f_Win_110250_75/fold1/rock/856_15.wav
NONE - Predicted genre: jazz
BIM - Predicted genre: country
FGSM - Predicted genre: country

j: 6933 gtzan_3f_Win_110250_75/fold1/rock/859_

NONE - Predicted genre: metal
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 6985 gtzan_3f_Win_110250_75/fold1/rock/852_17.wav
NONE - Predicted genre: rock
BIM - Predicted genre: reggae
FGSM - Predicted genre: jazz

j: 6986 gtzan_3f_Win_110250_75/fold1/rock/858_21.wav
NONE - Predicted genre: rock
BIM - Predicted genre: disco
FGSM - Predicted genre: disco

j: 6987 gtzan_3f_Win_110250_75/fold1/rock/844_19.wav
NONE - Predicted genre: rock
BIM - Predicted genre: disco
FGSM - Predicted genre: disco

j: 6988 gtzan_3f_Win_110250_75/fold1/rock/845_19.wav
NONE - Predicted genre: country
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 6989 gtzan_3f_Win_110250_75/fold1/rock/853_1.wav
NONE - Predicted genre: country
BIM - Predicted genre: pop
FGSM - Predicted genre: pop

j: 6990 gtzan_3f_Win_110250_75/fold1/rock/833_15.wav
NONE - Predicted genre: classical
BIM - Predicted genre: jazz
FGSM - Predicted genre: jazz

j: 6991 gtzan_3f_Win_110250_75/fold1/rock/861_15.wav
NONE 

NONE - Predicted genre: rock
BIM - Predicted genre: disco
FGSM - Predicted genre: disco

j: 7043 gtzan_3f_Win_110250_75/fold1/rock/864_21.wav
NONE - Predicted genre: jazz
BIM - Predicted genre: country
FGSM - Predicted genre: hiphop

j: 7044 gtzan_3f_Win_110250_75/fold1/rock/860_1.wav
NONE - Predicted genre: jazz
BIM - Predicted genre: country
FGSM - Predicted genre: pop

j: 7045 gtzan_3f_Win_110250_75/fold1/rock/843_13.wav
NONE - Predicted genre: disco
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 7046 gtzan_3f_Win_110250_75/fold1/rock/860_3.wav
NONE - Predicted genre: country
BIM - Predicted genre: jazz
FGSM - Predicted genre: jazz

j: 7047 gtzan_3f_Win_110250_75/fold1/rock/842_9.wav
NONE - Predicted genre: rock
BIM - Predicted genre: disco
FGSM - Predicted genre: disco

j: 7048 gtzan_3f_Win_110250_75/fold1/rock/834_17.wav
NONE - Predicted genre: rock
BIM - Predicted genre: reggae
FGSM - Predicted genre: jazz

j: 7049 gtzan_3f_Win_110250_75/fold1/rock/842_17.wav
NONE -

NONE - Predicted genre: reggae
BIM - Predicted genre: pop
FGSM - Predicted genre: hiphop

j: 7101 gtzan_3f_Win_110250_75/fold1/rock/864_11.wav
NONE - Predicted genre: jazz
BIM - Predicted genre: country
FGSM - Predicted genre: pop

j: 7102 gtzan_3f_Win_110250_75/fold1/rock/862_10.wav
NONE - Predicted genre: rock
BIM - Predicted genre: country
FGSM - Predicted genre: country

j: 7103 gtzan_3f_Win_110250_75/fold1/rock/854_9.wav
NONE - Predicted genre: reggae
BIM - Predicted genre: pop
FGSM - Predicted genre: hiphop

j: 7104 gtzan_3f_Win_110250_75/fold1/rock/845_8.wav
NONE - Predicted genre: country
BIM - Predicted genre: rock
FGSM - Predicted genre: rock

j: 7105 gtzan_3f_Win_110250_75/fold1/rock/849_1.wav
NONE - Predicted genre: rock
BIM - Predicted genre: metal
FGSM - Predicted genre: metal

j: 7106 gtzan_3f_Win_110250_75/fold1/rock/834_16.wav
NONE - Predicted genre: metal
BIM - Predicted genre: pop
FGSM - Predicted genre: pop

j: 7107 gtzan_3f_Win_110250_75/fold1/rock/843_14.wav
NONE 